In [1]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera(fps=20)
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
# display(image_widget)

from jetbot import Robot
robot = Robot()

In [2]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.0, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
brake_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.0, description='braking')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, brake_slider, steering_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='braking', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [3]:
import cv2
import numpy as np
import math

def maskImage(image):
    """ Method to define a region of interest with masking an image. 
        In(1): image - The image that should be masked 
        In(2): heightPercent - The percent of the height that should be masked
        Out(1): returns a image where half of the image is masked with a black surface
    """
    height, width = image.shape
    mask = np.zeros_like(image)

    # A polygon defining the region that should be masked as np.array
    polygon = np.array([[
        (0, height * 0.0),  # Up left
        (width, height * 0.0),  # 
        (width, height),
        (0, height),
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(image, mask)

def findCenterline(image):
    slope = 0.0

    # Apply gaussian image blurring
    gaussKernelSize = (3, 3)
    blurred_img = cv2.GaussianBlur(image, gaussKernelSize, 0)
    
    # Convert to HSV image representation
    hsv_img = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Filter out yellow of the center line
    low_yellow = np.array([5, 40, 100])
    up_yellow = np.array([50, 255, 255])
    col_img = cv2.inRange(hsv_img, low_yellow, up_yellow)
    
    # Filter out region of interest
    region_img = maskImage(col_img)
    
    # Apply canny edge detection
    canny_img = cv2.Canny(region_img, 100, 150)
    
    lines = cv2.HoughLinesP(canny_img, rho=1, theta=np.pi/180, threshold=30, lines=np.array([]), minLineLength=5, maxLineGap=50)
    if np.any(lines) == None:
        pass
    else:
        # Center line detection
        lines_x = []
        lines_y = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                lines_x.extend([x1, x2])
                lines_y.extend([y1, y2])

        min_y = int(image.shape[0] * 0.0)
        max_y = image.shape[0]  # <-- The bottom of the image
        
        poly = np.poly1d(np.polyfit(lines_y, lines_x, deg=1))
        center_x_start = int(image.shape[1] * 0.5)  # start in the middle of the picture
        center_x_end = int(poly(min_y))

        if (center_x_end - center_x_start) == 0:
            slope = 0
        else:
            slope = round(np.rad2deg(np.arctan2((max_y - min_y),
                          (center_x_start - center_x_end))) - 90, 3)
    return slope

In [4]:
angle = 0.0
angle_last = 0.0
angle_history = []
import time

def execute(change):
    global angle, angle_last, angle_history
    image = change['new']
    
    target_speed = speed_gain_slider.value
    
    angle = findCenterline(image)
    angle = np.deg2rad(angle)
    

    # Speed variational control
    #angle_history.append(angle)
    #if len(angle_history) >= 5:
    #    angle_history.pop(0)
    #brake = brake_slider.value * np.sum(np.abs(angle_history))
    #angle_last = angle
    
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value # + brake
    
    robot.left_motor.value = max(min(target_speed + pid, 1.0), 0.0)
    robot.right_motor.value = max(min(target_speed - pid, 1.0), 0.0)
    
    steering_slider.value = pid  # Just visualize steering
    
execute({'new': camera.value})

In [5]:
camera.observe(execute, names='value')

In [ ]:
### LINE TO AVOID KILLING ###

In [6]:
import time
camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()